# <center>TITRE - Projet Python</center>

## Membres:
* Corentin Drezen - 40007337, TD2
* Nathanael NOM - N°ETUDIANT, TD1
* Irfane NOM - N°ETUDIANT, TD2

## Introduction:
On souhaite réaliser un jeu du sytle <i>shoot them up</i> en 2D à l'aide du module $\color{#B267E6}{\text{pygame}}$ avec le personnage principal qui suit la souris

<img src="SpaceInvaders-Gameplay.gif">


In [ ]:
import pygame

## Timers
Le jeu a besoin de gérer plusieurs delais afin de réaliser le rendu graphique et certaines mécaniques.

Les jeux pygame fonctionent avec une boucle `while` 'infinie' car le systeme d'évenements est uniquement utilisable avec une fonction `pygame.event.get()` qui retourne une liste d'évenements.

In [ ]:
def main():
    #...
    while leJeuTourne:
        #...
        for event in pygame.event.get():
            #...

Le programme va s'éxécuter aussi vite qu'il peut mais un écran ne peut afficher que 60 à 144 images/secondes aujourd'hui et les appels au fonctions graphique incéssantes pouraient ralentir le jeu. Donc on a besoin d'un délai mais on ne veut pas bloquer le programme.

Alors on utilise les timers intégrés dans le module pygame qui fonctionent en parallèle et peuvents être utilisé avec son sytème d'évenements

<br>
On commence par définir les identifiants de nos propres évenements:

In [ ]:
AFFICHAGE_TIMER_EVENT = pygame.USEREVENT + 0 #évenement pour l'affichage
DELAY_TIRE_EVENT = pygame.USEREVENT + 2 #évenement pour avoir un délai entre chaque tir du personnage
SPAWN_TIMER_EVENT = pygame.USEREVENT + 3 #évenment pour l'apparition des ennemis

D'après le wiki il faut que ce soit des `int` compris entre `pygame.USEREVENT` (=24) et `pygame.NUMEVENTS - 1` (=32). - *"User defined events should have a value in the inclusive range of USEREVENT to NUMEVENTS - 1. It is recommended all user events follow this system."*

On doit ensuite appeler `pygame.time.set_timer(`***identifiant*** (int)`,` ***délai*** (int)`,` optionel: ***une_fois***=False`)` là où on souhaite lancer le timer. Elle déclanchera un évenement après chaque 'pauses'.

In [ ]:
def main():
    #...
    pygame.time.set_timer(AFFICHAGE_TIMER_EVENT, int(1000/fps)) 

Ici on utlise `int(1000/`**fps**`)` comme valeur pour le délai afin d'avoir une valeur relative à 'fps' (pour frames par secondes). Si 'fps' est égal à 60 on obtient 60 évenements `AFFICHAGE_TIMER_EVENT` par secondes: 1 toutes les ~17 ms.

Dans la boucle qui itère parmis tout les évenements on 'filtre' notre `AFFICHAGE_TIMER_EVENT` et on peut ainsi appelé notre fonction d'affichage/dessin/'peinture' à intervale régulier.

In [ ]:
#...
while leJeuTourne:
    #...
    for event in pygame.event.get():
        
        if event.type == AFFICHAGE_TIMER_EVENT:
            #...changer quelque chose juste avant le rafraichisement..
            Affichage()

## Affichage

## Déplacements

Les déplacements des acteurs du jeu son gérés dans leur `class` (Nafaire) avec la fonction `déplacement(`***x*** (int)`,` ***y*** (int)`)`.

`class` Nafaire:


In [ ]:
class Nafaire:
    def __init__(self, position=(0,0), animation=None, (...) , vitesseX=0.1, vitesseY=0.1, rect=None):
        
        self.x , self.y = position
        self.anciennePos = position
        self.animation = animation
        #...   
        self.vitesseX = vitesseX
        self.vitesseY = vitesseY    
        #...

    def deplacement(self, x, y):
        #...

    def detectCollisions(self):
        #...

Cette `class` a plusieurs propriétés et arguments:
- `position` (tuple), de simples coordonées dans la fenêtre du jeu. Elles sont ensuite distribuées dans les propriétés ***x*** et ***y***
- `anciennePos` (tuple), elle sert à garder l'ancienne position en mémoire après un déplacement. Elle a la même valeur que `position` à l'initialisation de l'objet.
- `vitesseX` (float) et `vitesseY` (float), ce ne sont pas vraiment des vitesses de physique '*v = d/t*' mais des multiplicateurs qui permettent un déplacement plus ou moins conséquents.
- `rect` (pygame.Rect), un type d'objet qui sert a dessiner une image rectangulaire avec les propriétés comme ***haut***, ***bas***, ***gauche***, ***droite*** ou encore ***centre*** avec pygame.
- `animation` (list), une liste de pygame.Surface (images), chaque frames d'animation de l'acteur. Leurs dimentions doivent être constantes.

Ces valeurs sont utilisées dans la fonction `déplacement(`***x*** (int)`,` ***y*** (int)`)`:

In [ ]:
def deplacement(self, x, y):
        self.anciennePos = (self.x, self.y)
        self.x += x * self.vitesseX
        self.y += y * self.vitesseY
        self.rect = self.animation[0].get_rect(center=(self.x, self.y))

        self.detectCollisions()

A chaque fois que la fonction est appelée les arguments ***x*** et ***y***, qui peuvent avoir des valeurs négatives, sont multipliés par leurs 'vitesses' respectives puis ajoutés aux propriétés ***self.x*** et ***self.y***. Ce qui donne l'illusion de mouvement car cette fonction est appelée très régulièrement et met à jour les valeurs utilisées par la fonction d'affichage. 

La propriété ***rect*** est mis à jour avec la fonction `get_rect()` de l'objet `pygame.Surface` (image dans pygame). C'est elle qui servira à placer l'acteur lors de l'affichage car ce type d'objet est plus pratique que de simples coordonnées. Sa propriété ***centre*** permettra de placer l'acteur sur un point à partir de son centre facilement au lieu le placer à partir de son 'haut gauche'.

L'ancienne position est conservé lors du déplacement car elle servira à gérer les collisions.

La fonction `.detectCollisions()` qui sert a trouver des collisions est appelée ici car c'est seulement après un déplacement qu'il peut y avoir une collisions avec un autre acteur.

## Colisions
Afin d'empécher que les acteurs du jeu se traversent et que le joueur puisse sortir de la scène il faut un système de collision.

pygame ne gère pas les collisions mais le type `pygame.Rect` possède une fonction `.colliderect(`***rect*** (pygame.Rect)`)` qui permet de savoir si un rectangle est en parti ou entièrement dans un autre rectangle.

In [ ]:
def detectCollisions(self):

        if(self.x < 0 or self.x > dimentions.x or self.y < 0 or self.y > dimentions.y): 
            pygame.event.post(pygame.event.Event(COLLISION_EVENT, source=self, collision=None))        

        if(self.type == NafaireTypes.JOUEUR):
            
            for enemi in enemies:
                if self.rect.colliderect(enemi.rect): 
                    pygame.event.post(pygame.event.Event(COLLISION_EVENT, source=self, collision=enemi))

        elif(self.type == NafaireTypes.ENNEMI and self.rect.colliderect(Joueur.rect)): 
            pygame.event.post(pygame.event.Event(COLLISION_EVENT, source=self, collision=Joueur))

        elif(self.type == NafaireTypes.BALLE):
            
            for enemi in enemies:
                
                if self.rect.colliderect(enemi.rect): 
                    pygame.event.post(pygame.event.Event(COLLISION_EVENT, source=self, collision=enemi))
                    break

        elif(self.type == NafaireTypes.BALLE_ENNEMI and self.rect.colliderect(Joueur.rect)): 
            pygame.event.post(pygame.event.Event(COLLISION_EVENT, source=self, collision=Joueur))


In [ ]:
#...
for event in pygame.event.get():

            #...gestion d'autres évenements...
            
            elif event.type == COLLISION_EVENT:

                if(event.collision == None):#collision avec la 'bordure du cadre'
                
                    if(event.source.type == NafaireTypes.JOUEUR or event.source.type == NafaireTypes.ENNEMI):
                        ###corrige la position du joueur ou de l'ennemi pour qu'il ne sorte pas du cadre
                        if event.source.x < 0 : event.source.x = 0
                        if event.source.x > dimentions.x : event.source.x = dimentions.X
                        if event.source.y < 0 : event.source.y = 0
                        if event.source.y > dimentions.y : event.source.y = dimentions.y

                    elif(event.source.type == NafaireTypes.BALLE):
                        ###détruit les balles hors champs
                        if event.source in balleList:
                            balleList.remove(event.source)                

                elif(event.source.type == NafaireTypes.BALLE and event.collision.type == NafaireTypes.ENNEMI):#collision balle ennemi
                    if event.collision in enemies:
                        enemies.remove(event.collision)
                    if event.source in balleList:
                        balleList.remove(event.source)
                    score += 1
                
                elif(event.source.type == NafaireTypes.ENNEMI and event.collision.type == NafaireTypes.JOUEUR) or (event.source.type == NafaireTypes.JOUEUR and event.collision.type == NafaireTypes.ENNEMI):#collision joueur ennemi
                    event.source.x, event.source.y = event.source.anciennePos
                    event.source.rect = event.source.animation[0].get_rect(center=(event.source.x, event.source.y))

## Tires

In [ ]:
def tire():
    global peutTirer
    peutTirer = False
    balle = Nafaire([Joueur.rect.centerx + 4, Joueur.rect.top - 22], balleImg, type=NafaireTypes.BALLE, vitesseY=8)
    balleList.append(balle)

    pygame.time.set_timer(DELAY_TIRE_EVENT, int(1000/tireParSecondes), True)
